# Analysis of sold cars in Estonia

Data is taken from official source - https://www.transpordiamet.ee/soidukitega-tehtud-toimingute-statistika

Period - 5 years.

Only new cars where used in this analysis.


In [23]:
%reload_ext autoreload
%autoreload 0

import os

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [24]:
from mnt_sum import get_summary, COLUMNS, COLUMN_SHORT_NAME, COLUMN_REG_DATE, COLUMN_CUSTOMER, PRIVATE_CUSTOMER, COLUMN_ENGINE_TYPE

df_o = pd.concat([get_summary(f"data/{year}") for year in range(2018, 2024)])
df_o = df_o[COLUMNS +[COLUMN_REG_DATE]]
df_o.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99580 entries, 42 to 1532
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Mark             99580 non-null  object 
 1   Mudel            99580 non-null  string 
 2   short name       99580 non-null  object 
 3   Mootori tüüp     99580 non-null  object 
 4   Mootori maht     99580 non-null  Int64  
 5   Mootori võimsus  99580 non-null  Float64
 6   Linn             99580 non-null  object 
 7   Tüüp (isik)      99580 non-null  object 
 8   Arv              99580 non-null  Int64  
 9   Värv             59188 non-null  string 
 10  Esm reg aasta    99580 non-null  Int16  
dtypes: Float64(1), Int16(1), Int64(2), object(5), string(2)
memory usage: 8.9+ MB


In [25]:
df_o.head(5)

,Mark,Mudel,short name,Mootori tüüp,Mootori maht,Mootori võimsus,Linn,Tüüp (isik),Arv,Värv,Esm reg aasta
42,ALFA ROMEO,GIULIA,ALFA ROMEO GIULIA,BENSIIN_KATALYSAATOR,1995,206.0,Määramata,JURIIDILINE,1,<NA>,2018
43,AUDI,A1 SPORTBACK,AUDI A1,BENSIIN_KATALYSAATOR,1395,110.0,Tallinn,FÜÜSILINE,1,<NA>,2018
44,AUDI,A4 AVANT,AUDI A4,BENSIIN_KATALYSAATOR,1984,185.0,Määramata,FÜÜSILINE,1,<NA>,2018
45,AUDI,A4 AVANT,AUDI A4,DIISEL,1968,140.0,Tallinn,JURIIDILINE,1,<NA>,2018
46,AUDI,A4 LIMOUSINE,AUDI A4,BENSIIN_KATALYSAATOR,1395,110.0,Tartu,FÜÜSILINE,1,<NA>,2018


In [26]:

df_models =  df_o.groupby(
        [COLUMN_SHORT_NAME], 
        as_index=False
    )['Arv'].sum().sort_values('Arv', ascending=False).reset_index(drop=True)
df_models.head()

,short name,Arv
0,TOYOTA RAV4,7754
1,SKODA OCTAVIA,6046
2,TOYOTA COROLLA,5485
3,SKODA KODIAQ,3581
4,RENAULT CLIO,3580


In [27]:
df_models_year =  df_o.groupby(
        [COLUMN_SHORT_NAME, COLUMN_REG_DATE], 
        as_index=False
    )['Arv'].sum().sort_values('Arv', ascending=False).reset_index(drop=True)

df_models_year.head()

,short name,Esm reg aasta,Arv
0,TOYOTA RAV4,2022,1526
1,TOYOTA RAV4,2021,1440
2,TOYOTA RAV4,2020,1400
3,TOYOTA RAV4,2023,1311
4,TOYOTA RAV4,2019,1194


# Marks overview

In [28]:
df_marks_year =  df_o.groupby(
        ['Mark', COLUMN_REG_DATE], 
        as_index=False
    )['Arv'].sum().sort_values('Arv', ascending=False).reset_index(drop=True)

df_marks_year.head()

,Mark,Esm reg aasta,Arv
0,TOYOTA,2022,4456
1,TOYOTA,2023,4211
2,TOYOTA,2021,3851
3,TOYOTA,2018,3665
4,TOYOTA,2019,3444


In [29]:
from matplotlib.colors import LinearSegmentedColormap


def sort_and_plot(_df, sortby=2023, limit=20, main_groupby=COLUMN_SHORT_NAME):
    cm = LinearSegmentedColormap.from_list(
        name="ryg",
        colors=["red", "yellow", "green"],
    )

    df_group = (
        _df.groupby([main_groupby, COLUMN_REG_DATE], as_index=False)["Arv"]
        .sum()
        .sort_values("Arv", ascending=False)
        .reset_index(drop=True)
    )

    df_h = (
        df_group.pivot(index=main_groupby, columns=COLUMN_REG_DATE, values="Arv")
        .sort_values([sortby], ascending=False)
        .astype("float64")
    )

    df_h = df_h.head(limit)

    df_h.style.format(precision=0)
    df_colorized = df_h.style.background_gradient(cmap=cm).format(precision=0)

    return df_colorized


sort_and_plot(df_marks_year, main_groupby="Mark")

Esm reg aasta,2018,2019,2020,2021,2022,2023
Mark,,,,,,
TOYOTA,3665,3444,3362,3851,4456,4211
SKODA,2840,3079,2884,3157,2769,3358
AUDI,385,390,496,539,1000,2544
KIA,1590,1529,1259,1867,1636,1813
VOLKSWAGEN,1690,1726,1197,1968,1595,1228
PEUGEOT,1128,1055,1173,1393,816,880
RENAULT,2592,2409,2021,1152,784,873
DACIA,943,1080,706,742,725,750
SUBARU,794,798,347,612,442,746


# Total overview

In [30]:
sort_and_plot(df_models_year)

Esm reg aasta,2018,2019,2020,2021,2022,2023
short name,,,,,,
TOYOTA RAV4,883,1194,1400,1440,1526,1311
TOYOTA COROLLA,378,1091,977,922,966,1151
SKODA OCTAVIA,911,1031,1054,1110,888,1052
SKODA KODIAQ,462,675,536,480,691,737
AUDI Q2,30,24,18,17,229,736
AUDI A3,26,12,11,15,34,642
KIA SPORTAGE,602,571,399,633,641,599
KIA CEED,533,511,403,664,512,553
TOYOTA YARIS CROSS,nan,nan,nan,49,498,510


# Private owners

In [31]:
df_private = df_o[df_o[COLUMN_CUSTOMER].apply(lambda x: PRIVATE_CUSTOMER in x)]

sort_and_plot(df_private)

Esm reg aasta,2018,2019,2020,2021,2022,2023
short name,,,,,,
TOYOTA RAV4,557,736,694,736,783,771
TOYOTA COROLLA,203,469,458,431,301,453
SKODA OCTAVIA,420,482,598,492,289,373
SKODA KODIAQ,266,390,330,249,321,350
TOYOTA YARIS CROSS,nan,nan,nan,22,293,348
KIA SPORTAGE,350,287,202,405,350,294
SUBARU OUTBACK,240,203,99,205,174,270
SKODA KAMIQ,nan,45,242,187,131,228
SKODA KAROQ,269,294,170,230,151,208


# Private owners electric cars

In [32]:
df_private_e = df_private[df_private[COLUMN_ENGINE_TYPE] == 'ELEKTER']

sort_and_plot(df_private_e)

Esm reg aasta,2018,2019,2020,2021,2022,2023
short name,,,,,,
TESLA MODEL Y,nan,nan,nan,1,1,63
SKODA ENYAQ,nan,nan,nan,22,27,59
KIA EV6,nan,nan,nan,nan,3,25
HYUNDAI IONIQ5,nan,nan,nan,4,11,22
VOLKSWAGEN ID.4,nan,nan,nan,6,3,19
TESLA MODEL 3,nan,nan,11,5,11,18
NISSAN LEAF,10,19,23,7,11,17
AUDI Q4,nan,nan,nan,nan,1,17
NISSAN ARIYA,nan,nan,nan,nan,nan,15


# Private owners hybrid cars

In [33]:
df_private_h = df_private[df_private[COLUMN_ENGINE_TYPE].apply(lambda x: 'HYBRIID' in x)]

sort_and_plot(df_private_h)

Esm reg aasta,2018,2019,2020,2021,2022,2023
short name,,,,,,
TOYOTA COROLLA,nan,207,213,276,187,427
TOYOTA RAV4,279,314,291,347,357,412
TOYOTA YARIS CROSS,nan,nan,nan,13,203,330
KIA SPORTAGE,nan,nan,1,29,222,247
TOYOTA C-HR,177,155,120,218,196,153
HONDA CR-V,nan,76,79,179,210,152
SKODA OCTAVIA,nan,nan,30,135,67,144
NISSAN QASHQAI,nan,nan,nan,78,156,143
HYUNDAI TUCSON,nan,nan,8,151,177,133
